In [2]:
import os
import pathlib
import pandas as pd
import numpy as np
import glob
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.SeqIO.FastaIO import SimpleFastaParser
import subprocess 
from BCBio import GFF
import re 
import matplotlib.pyplot as plt 

In [3]:
#Fasta file writer 
def FastaFile(gene_id,seq,outname):
    initial_path = os.getcwd()
    with open(outname,'a') as f:
        f.write(">"+gene_id+"\n"+seq+"\n") 
    os.chdir(initial_path)


def extract_upstream_sequence(contig_id, start, end,genome_dict, upstream_length=3000):
    """
    Extracts the sequence upstream of a gene given its genomic location.
    contig_id: str, ID of the contig containing the gene.
    start: int, 1-based start position of the gene.
    end: int, 1-based end position of the gene.
    upstream_length: int, length of sequence to extract upstream of the gene.
    Returns: str, upstream sequence of the gene.
    """

    # Get sequence object for contig
    contig_seq = genome_dict[contig_id].seq
    # Compute start and end positions of upstream sequence
    upstream_start = max(start - upstream_length, 0)
    upstream_end = start - 1
    # Extract upstream sequence
    upstream_seq = contig_seq[upstream_start:upstream_end]
    return str(upstream_seq)

In [5]:
#Class Class()
class Class():
    def load(self,filename): #load proteome data (single or multiple fasta files) 
        # Create an empty DataFrame with the desired column names
        self.input_seq_db  = pd.DataFrame(columns=["gene_id", "seq", "len"])
        # Open the FASTA file and iterate over its records
        with open(filename) as in_handle:
            for title, seq in SimpleFastaParser(in_handle):
                # Add the current record to the DataFrame
                len_seq = len(seq)
                gene_id = title.split()[0]
                self.input_seq_db = pd.concat([self.input_seq_db, pd.DataFrame([[gene_id, seq, len_seq]], columns=["gene_id", "seq", "len"])])
        self.input_seq_db.set_index(keys=self.input_seq_db["gene_id"],inplace=True)
        return self 
    def combine_df(self,filename): #load proteome data (single or multiple fasta files) 
        with open(filename) as in_handle:
            for title, seq in SimpleFastaParser(in_handle):
                # Add the current record to the DataFrame
                len_seq = len(seq)
                gene_id = title.split()[0]
                self.input_seq_db = pd.concat([self.input_seq_db, pd.DataFrame([[gene_id, seq, len_seq]], columns=["gene_id", "seq", "len"])])
        return self 

    def hmmer_cyp(self,input_fasta,):
        # Run HMMER
        subprocess.check_output(['hmmsearch', "-E","0.1", "--tblout", "hmm_hits.txt","Pfam-A.hmm",input_fasta])
        # Process the HMMER output and add hits to the DataFrame
        with open("hmm_hits.txt") as f:
            for line in f:
                if not line.startswith('#'):
                    fields = line.strip().split()
                    e_value = fields[4]
                    gene_id = fields[0]
                    domain_name = fields[3]
                    if domain_name in ['PF00067.25', 'PF00067']:
                        mask = self.input_seq_db["gene_id"] == gene_id
                        if mask.any():
                            self.input_seq_db.loc[mask, "Hmmer_CYP"] = e_value
        # Remove the HMMER output file
        os.remove("hmm_hits.txt")
        return self 

 
    def blastp_comparison(self, query, subject):
        # Run blastp
        subprocess.check_output(['blastp', "-query", query,"-subject",subject, "-out","blastp_output.txt","-outfmt","6","-max_target_seqs","1","-evalue","0.1"])
        # Process the blastp output and add hits to the DataFrame
        with open('blastp_output.txt') as f:
            for line in f:
                fields = line.split()
                gene_id = fields[0]
                subject_id = fields[1]
                percentage = fields[2]
                e_value = fields[10]
                mask = self.input_seq_db["gene_id"] == gene_id
                header = subject_id+"aa identity"
                if mask.any():
                    self.input_seq_db.loc[mask, header] = str(percentage)
                    self.input_seq_db.loc[mask, "evalue"] = str(e_value)
        os.remove("blastp_output.txt")
        return self
    
    def blastp_label(self,query,subject):
        # Run blastp of the transcriptome object against a target protein, label the e-value and %homology of hits 
        subprocess.check_output(['blastp', "-query", query,"-subject",subject, "-out","blastp_output.txt","-outfmt","6","-max_target_seqs","1","-evalue","0.1"])
        # Process the blastp output and add hits to the DataFrame
        with open('blastp_output.txt') as f:
            for line in f:
                fields = line.split()
                gene_id = fields[0]
                subject_id = fields[1]
                percentage = fields[2]
                e_value = fields[10]
                mask = self.input_seq_db["gene_id"] == gene_id
                if mask.any():
                    self.input_seq_db.loc[mask, "CYP blastp - CYP Name"] = subject_id
                    self.input_seq_db.loc[mask, "CYP blastp - aa perc"] = percentage
                    self.input_seq_db.loc[mask, "CYP blastp - e value"] = e_value
        os.remove("blastp_output.txt")
        return self  
    
    def get_distribution_identity(self,header_name):
        #Use this after using blastp_label to plot the results 
        plt.hist(self.input_seq_db[header_name].str.split(",").str[0].astype(float), bins=20, range=(0, 100))
        plt.xlabel("Percentage Amino Acid Identity")
        plt.ylabel("Count")
        plt.title("Distribution of Percentage Amino Acid Identity")
        plt.show()
        return plt 

In [29]:
tacca = Class() 
tacca.load("tchantrieri_proteome.fasta")

with open("hmm_hits_tps.txt") as f:
    for line in f:
        if not line.startswith('#'):
            fields = line.strip().split()
            e_value = fields[4]
            gene_id = fields[0]
            domain_name = fields[3]
            if domain_name in ['PF00432.24']:
                mask = tacca.input_seq_db["gene_id"] == gene_id
                if mask.any():
                    tacca.input_seq_db.loc[mask, "Hmmer_prenyltr"] = e_value

In [ ]:
import requests
from bs4 import BeautifulSoup

# Define a function that checks whether there are SRA experiments available for a given taxon
def check_sra(txid, seq_type):
    url = f"https://www.ncbi.nlm.nih.gov/sra?term={txid}%20{seq_type}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.find("title").text
    print(title)
    if "No items found" in title:
        return "No"
    else: 
        print('hi')
        return "Yes"


# Read in the CSV file and specify "Plant Taxonomy" as the column label
df = pd.read_csv('unique_plant_species.csv')
print(df)

# Add a new column for DNA-seq experiments and populate it with "Yes" or "No"
df['DNA-seq'] = df['Plant Species'].apply(lambda x: check_sra(x, "DNA"))

# Add a new column for RNA-seq experiments and populate it with "Yes" or "No"
df['RNA-seq'] = df['Plant Species'].apply(lambda x: check_sra(x, "RNA"))

# Save the updated DataFrame to a new CSV file
df.to_csv('sra_experiments.csv', index=False)

In [ ]:
#BLASTING AGAINST NELSON MOSS DB 
path_input = "/Users/bustin/Desktop/capstone/MOSSBIOINFO/transcriptomics/march25/biojustin/data/datadep"
os.chdir(path_input)
nelson = BioBustin()
nelson.load(filename="cyp_filtered_6000aa_wref.fasta",path_input=path_input)
for index,row in nelson.input_seq_db.iterrows():
    seq = row["seq"]
    query = row["gene_id"]
    outname = subject+".fasta"
    FastaFile(gene_id=query,seq=seq,outname= outname)
    nelson.blastp_label(query=outname, subject="cyp_db_full.fasta")
    os.remove(outname)


In [ ]:
nelson.input_seq_db.loc[:,"CYP blastp - CYP Name":"CYP blastp - e value]"].to_csv("nelson_label_6000.csv")

In [ ]:
path_input = "/Users/bustin/Desktop/capstone/MOSSBIOINFO/transcriptomics/march25/biojustin/data/datadep"

test1 = BioBustin()
test1.load(filename="cyp_filtered_6000aa_wref.fasta",path_input=path_input)
test1.hmmer_cyp(pfam="PF00067.hmm",input_fasta="cyp_filtered_6000aa_wref.fasta",path_input=path_input)
test1.blastp_comparison(query="cyp_filtered_6000aa_wref.fasta",subject="CpCYP964A1_p.fasta")
test1.EC_CLEAN(input_file="cyp_filtered_6000aa_wref.fasta")
print(test1.input_seq_db)

In [ ]:
#Make the full nelson cyp database fasta files 
with open("cyp_db_full.fasta",'a') as f:
    for i in ["Chlorophyte_algae","Charophyte_algae","MossRaw","Liverworts","Hornworts","Lycophytes","Fern","Gymnosperm"]:
        df = pd.read_excel("nelson_cyp_db.xlsx", i)
        for index, row in df.iterrows():
            seq = row["Sequence"]
            gene_id = str(row["CYP name"]) + "," + str(row["Species"]) + "," + i
            f.write(">"+gene_id+"\n"+seq+"\n")


In [ ]:
#Make just moss db 
xls = pd.ExcelFile('NELSON_DB.xlsx')
df = pd.read_excel(xls, "MossRaw")
with open("cyp_db_moss.fasta",'w') as f:
    for index, row in df.iterrows():
        seq = row["Sequence"]
        gene_id = row["CYP name"] + "," + row["Species"] + "," + "MossRaw"
        f.write(">"+gene_id+"\n"+seq+"\n")
        


In [ ]:
#OS STUFF PATH STUFF 
#print(os.getcwd())
#fasta_file = "CpCYP964A1_p.fasta"
#print(os.path.isfile(fasta_file))
#st = os.stat(fasta_file)
#print(st)